# 05: Abstraktní metody, rozhraní a implementace dispose
**autor: Erik Král ekral@utb.cz**

---


Obsah
- Abstraktní metoda a třída.
- Rozhraní.
- Příklad implementace rozhraní IDisposable.



## Abstraktní metoda a třída

Abstraktní metoda nemá implementaci (tělo) a může být definována pouze v abstraktní třídě. Abstraktní třída slouží pouze jako rodičovská třída a v klientském kódu používáme její potomky, ale nemůžeme vytvářet její instance. Opět jde o konstrukci kterou vytváříme z důvodu statické typové kontroly.

Nyní si projdeme příklad, kdy si nejprve vytvoříme virtuální metodu `VratZvuk`, kterou poté změníme na abstraktní.

Nejprve si definujeme třídu `Zviratko`, ktera ma property `Jmeno` a virtuální metodu `VratZvuk`. A potom definujeme dva potomky této třídy, které překryjí s použitím klíčového slova `override` metodu `VratZvuk`. Prvním potomkem je třída `Pejsek`, která vrací zvuk "haf haf" a druhým třída `Kocicka`, která vrací zvuk "mnau".

In [2]:
class Zviratko
{
    public string Jmeno { get; set; }

    public virtual string VratZvuk()
    {
        return "nema zadny konkretni zvuk";
    }
}

class Pejsek : Zviratko
{
    public override string VratZvuk()
    {
        return "haf haf";
    }
}

class Kocicka : Zviratko
{
    public override string VratZvuk()
    {
        return "mnau";
    }
}

In [3]:
Zviratko zviratko = new Zviratko() { Jmeno = "Obecne zviratko" };
Zviratko pejsek = new Pejsek() { Jmeno = "Azor" };
Zviratko kocicka = new Kocicka() { Jmeno = "Micka" };

Console.WriteLine($"{zviratko.Jmeno} dela {zviratko.VratZvuk()}");
Console.WriteLine($"{pejsek.Jmeno} dela {pejsek.VratZvuk()}");
Console.WriteLine($"{kocicka.Jmeno} dela {kocicka.VratZvuk()}");

Obecne zviratko dela nema zadny konkretni zvuk
Azor dela haf haf
Micka dela mnau


Pokud se zamyslíme nad metodou `VratZvuk` ve třídě `Zviratko`, tak pro ni **nemáme smysluplnou implementaci** protože jde o obecné (abstraktní) zvířátko které nedělá žádný zvuk. Takové metody pro které nemáme implementaci a které musí implementovat až potomci této třídy můžeme označit jako abstraktní klíčovým slovem `abstract`. Abstraktní metody mohou mít pouze abstraktní třídy, třída `Zviratko` tedy musí být označená klíčovým slovem `abstract`. Takové třídy jsou potom určené pouze pro dědičnost a nemůžeme vytvářet jejich instance ale jen mít reference. Potomci těchto tříd potom musí povinně implementovat abstraktní metody. Abstraktní třída může být jako abstraktní označena klíčovým slovem `abstract` i když nemá abstraktní metody.

In [5]:
abstract class Zviratko
{
    public string Jmeno { get; set; }

    public abstract string VratZvuk();
}

class Pejsek : Zviratko
{
    public override string VratZvuk()
    {
        return "haf haf";
    }
}

class Kocicka : Zviratko
{
    public override string VratZvuk()
    {
        return "mnau";
    }
}

Zviratko pejsek = new Pejsek() { Jmeno = "Azor" };
Zviratko kocicka = new Kocicka() { Jmeno = "Micka" };

Console.WriteLine($"{pejsek.Jmeno} dela {pejsek.VratZvuk()}");
Console.WriteLine($"{kocicka.Jmeno} dela {kocicka.VratZvuk()}"); 

Azor dela haf haf
Micka dela mnau


Použití tříd `Pejsek` a `Kocicka` a v klientském kódu zůstává stejné, jediný rozdíl je v tom, že nemůžeme vytvářet instance třídy `Zviratko`.

In [6]:
Zviratko zviratko = new Zviratko();

Error: (1,21): error CS0144: Cannot create an instance of the abstract type or interface 'Zviratko'

## Rozhraní
---

Rozhraní je podobné abstraktní třídě pouze s abstraktními metodami. Říkáme, že třída implementuje rozhraní. Každá třída může v jazyce c# dědit od jedné třídy ale může implementovat libovolný počet rozhraní. Rozhraní na rozdíl od abstraktních tříd neobsahují fieldy.

Z hlediska použití popisuje abstraktní třída vztah **"is a"** tedy pejsek **je** zvířátko nebo monitor **je** produkt. Zatímco rozhraní popisuje spíše vztah **"can do"** nebo možná lépe **"must do"**, tedy například že třída faktura **umí** serializaci do textového souboru nebo třída soubor **umí** metodu Dispose, tedy uvolnit všechny své alokované zdroje a zavřít otevřený soubor. Většinou preferujeme více jednoduchých rozhraní s méně metodami, než jedno velké rozhraní s mnoha metodami.

Rozhraní se používají často frameworcích kde pomocí nich určujeme co daná třída umí, například pomocí implementace rozhraní `IComparable` můžeme třídu naučit aby fungovala v metodě `Sort`. Rozhraní se také často používají v technice Dependency Injection, kterou probereme příště, kdy místo třídy používáme rozhraní a vlastní implementaci pak můžeme dle potřeb měnit, například místo reálné implementace použijeme testovací implementaci.

Nejprve si zopakujeme definici abstraktní třídy `Zviratko`.

In [7]:
abstract class Zviratko
{
    public abstract string VratZvuk();
}

class Pejsek : Zviratko
{
    public override string VratZvuk()
    {
        return "haf haf";
    }
}

Console.WriteLine(new Pejsek().VratZvuk());

haf haf


Podobný příklad bychom potom mohli zapsat pomocí rozhraní, které zápis zjednodušuje.

In [9]:
interface IZviratko
{
    string VratZvuk();
}

class Pejsek : IZviratko
{
    public string VratZvuk()
    {
        return "haf haf";
    }
}

Console.WriteLine(new Pejsek().VratZvuk());

haf haf


Vzhledem k tomu, že toto rozhraní by mohli implementovat i jiné třídy než zvířátka, tak bychom mohli toto rozhraní mohli také nazvat například `IZvuk` (anglicky `ISoundable`) a implementovat by ji mohla třeba i třída `Auto`.

In [11]:
interface IZvuk
{
    string VratZvuk();
}

class Pejsek : IZvuk
{
    public string VratZvuk()
    {
        return "haf haf";
    }
}

class Auto : IZvuk
{
    public string VratZvuk()
    {
        return "brmmm brmmm";
    }
}

Console.WriteLine(new Pejsek().VratZvuk());
Console.WriteLine(new Auto().VratZvuk());

haf haf
brmmm brmmm


Použití v klientském kódu potom může být následující.

In [12]:
static void VypisZvuk(IZvuk objektSeZvukem)
{
    Console.WriteLine(objektSeZvukem.VratZvuk());
}

Pejsek pejsek = new Pejsek();
Auto auto = new Auto();

VypisZvuk(pejsek);
VypisZvuk(auto);

haf haf
brmmm brmmm


V rozhraní můžou být i property. Pokud je properta v rozhraní, tak ji musíme v třídě implementovat.

In [ ]:
interface IRozhraniA
{
    int Properta { get; set; }
    void MetodaA();
}

class TridaA : IRozhraniA
{
    public int Properta { get; set; }

    public void MetodaA()
    {
        Console.WriteLine("MetodaA");
    }
}

Následující příklad demonstruje zjednodušené použití rozhraní IComparable.

In [ ]:
class MojeTrida : IComparable<MojeTrida>
{
    public int Cislo { get; set; }

    public int CompareTo(MojeTrida? other)
    {
        if (other == null) return 1;
        return this.Cislo.CompareTo(other.Cislo);
    }
}

MojeTrida[] pole = [new MojeTrida() { Cislo = 3 }, new MojeTrida() { Cislo = 7 }, new MojeTrida() { Cislo = 1 }];

Array.Sort(pole);

Console.WriteLine(string.Join(", ", pole.Select(x => x.Cislo)));

1, 3, 7


## Příklad implementace rozhraní IDisposable
---

Nyní si ukážeme školní příklad na implementaci rozhraní [IDisposable](https://learn.microsoft.com/en-us/dotnet/api/system.idisposable?redirectedfrom=MSDN&view=net-7.0). V následujícím kódu se nám po ukončení bloku using zavolá automaticky metoda `Dispose`.

In [13]:
class Trida : IDisposable
{
    public void Dispose()
    {
        Console.WriteLine("Uvolnuji zdroje..");
    }
}

using(Trida trida = new Trida())
{
    Console.WriteLine("Pracuji s tridou");
}


Pracuji s tridou
Uvolnuji zdroje..


Pokud bychom klíčové slovo using nepoužili, tak by se nám metoda Dispose nezavolá.

In [14]:
Trida trida = new Trida();

Console.WriteLine("Pracuji s tridou");

Pracuji s tridou


Následující příklad je pokročilý a pro splnění předmětu jej nemusíte studovat a nepopisuje kompletní problematiku uvolňování zdrojů. Příklad demonstruje implementaci rozhraní IDisposable, kde bereme v úvahu i to, že uživatel nepoužije Dispose a používáme i Finalizer (v C++ se mu říká destruktor.). Připomínám, že rozhraní IDisposable používáme jen ve speciálních případech, kdy musíme uvolnit alokované zdroje a běžně se v klientském kódu nepoužívá. Celá problematika interakce managed jazyka s unmanaged kódem složitá a nad rámec tohoto předmětu a více se můžete dočíst v dokumentaci [Implement a Dispose method](https://learn.microsoft.com/en-us/dotnet/standard/garbage-collection/implementing-dispose).

In [ ]:
class Trida : IDisposable
{
    System.Net.Http.HttpClient client = new();

    private bool disposed = false;

    public void Dispose()
    {
        Dispose(disposing: true);
        
        // Protoze jsem uz vse uvolnil, tak uz nechci aby se znovu volal Finalizer
        GC.SuppressFinalize(this);
    }

    protected virtual void Dispose(bool disposing)
    {
        if(!this.disposed)
        {
            if(disposing)
            {
                Console.WriteLine("Uvolnuji managed objekty");
                client.Dispose();
            }
            
            Console.WriteLine("Uvolnuji unmanaged objekty");
            // uvolnovani unmanaged objektu, kvuli kterym bych pripadne IDisposable musel implementovat
            
            disposed = true;
        }
    }

    // Finalizer, zavola se jako posledni moznost pred uvolneni objektu z pameti
    // Jako vhodnejsi alternativu pro uvolneni Unmanaged resources Microsoft doporucuje SafeHandle
    ~Trida()
    {
        Dispose(disposing: false);
    }
}

using(Trida trida = new Trida())
{
    Console.WriteLine("Pracuji s tridou");
}

#### Cvičení
---

1. Příklad na implementaci abstraktní třídy

Máte rodičovskou abstraktní třídu `Kontrolka`.

In [ ]:
abstract class Kontrolka
{
    public int sirka = 0;
    public int vyska = 0;

    public abstract void Vykresli();
}

Doplňte definici třídy `Tlacitko` tak,

In [ ]:
// definice tridy Tlacitko

Aby byl funkční následující kód a vypsal na terminál text "Kreslim tlacitko".

In [ ]:
Kontrolka kontrolka = new Tlacitko();
kontrolka.sirka = 200;
kontrolka.vyska = 300;

kontrolka.Vykresli(); 

2. Příklad na implementaci rozhraní

Máte rozhraní `IDrawable.

In [ ]:
interface IDrawable
{
    void Draw();
}

Definujte třídu `Obrazek` implementující rozhraní `IDrawable`, kdy v metode `Draw` vypiste na terminal "Kreslim obrazek"

In [ ]:
// Definice třídy obrázek

In [ ]:
IDrawable drawable = new Obrazek();
drawable.Draw();

3. V předchozích příkladech odstraňte definici rozhraní a abstraktní třídy a zkuste si jej sami nadefinovat.